Imports

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import plotly.graph_objects as go
import numpy as np

Read Codebook

In [2]:
with open("USCODE22_LLCP_102523.HTML", "r", encoding='latin1') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, "html.parser")

tables = soup.find_all("table", class_="table", attrs={"summary": "Procedure Report: Report"})

data_element_details = {}
for table in tables:
    data_element_detail = {}
    values = {}
    for i, row in enumerate(table.find_all("tr")):
        cells = [cell.get_text(strip=True, separator='\n').replace('\xa0', ' ') for cell in row.find_all(["td"])]
        if i == 0:
            matches = re.findall(r'(.*?):\s*([^\n]*)', cells[0])
            for match in matches:
                data_element_detail[match[0]] = match[1]
            if "SAS Variable Name" not in data_element_detail:
                break
        if i >= 2:
            values[cells[0]] = {"Value Label": cells[1], "Frequency": cells[2], "Percentage": cells[3], "Weighted Percentage": cells[4]}
    if "SAS Variable Name" in data_element_detail:
        data_element_detail['Valid Values'] = values
        data_element_details[data_element_detail["SAS Variable Name"]] = data_element_detail

Output as csv for manual labelling

In [3]:
pd.DataFrame.from_dict(data_element_details, orient="index").to_csv("fields.csv")

Read fixed width file data positions from html

In [4]:
url = 'CDC - BRFSS 2022 Combined Landline and Telephone Multiple Data.html'
tables = pd.read_html(url)
df_cols = tables[0]
column_names = df_cols['Variable Name']
start_positions = df_cols['Starting Column']
widths = df_cols['Field Length']
colspecs = list(zip(start_positions - 1, start_positions - 1 + widths))

Load file using fixed width positions extracted above

In [5]:
df = pd.read_fwf('LLCP2022.ASC', colspecs=colspecs, header=None, names=column_names)
num_cols = len(df.columns)
df['non_blank_count'] = df.isnull().sum(axis=1)
df['non_blank_perc'] = df['non_blank_count'] / (num_cols - 9)

In [6]:
df

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4,non_blank_count,non_blank_perc
0,1,1,2032022,2,3,2022,1100,2022000001,2022000001,1.0,...,2,0.0,1,0,1,1.0,2.0,2.0,177,0.558360
1,1,1,2042022,2,4,2022,1100,2022000002,2022000002,1.0,...,2,0.0,1,0,1,2.0,2.0,2.0,175,0.552050
2,1,1,2022022,2,2,2022,1100,2022000003,2022000003,1.0,...,2,0.0,1,0,1,NaN,NaN,2.0,171,0.539432
3,1,1,2032022,2,3,2022,1100,2022000004,2022000004,1.0,...,2,0.0,1,0,1,9.0,9.0,2.0,162,0.511041
4,1,1,2022022,2,2,2022,1100,2022000005,2022000005,1.0,...,1,10.0,1,140,1,NaN,NaN,2.0,176,0.555205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,78,11,12192022,12,19,2022,1100,2022001527,2022001527,NaN,...,7,900.0,9,99900,9,NaN,NaN,1.0,177,0.558360
445128,78,11,12212022,12,21,2022,1100,2022001528,2022001528,NaN,...,2,0.0,1,0,1,NaN,NaN,1.0,160,0.504732
445129,78,11,11292022,11,29,2022,1100,2022001529,2022001529,NaN,...,7,900.0,9,99900,9,2.0,2.0,2.0,169,0.533123
445130,78,11,12082022,12,8,2022,1100,2022001530,2022001530,NaN,...,2,0.0,1,0,1,1.0,1.0,1.0,165,0.520505


In [7]:
df = df.fillna(value='BLANK')
record_sum = len(df)

In [8]:
blank_counts = (df == "BLANK").sum()
with pd.option_context('display.max_rows', None):
    print(blank_counts)

_STATE                  0
FMONTH                  0
IDATE                   0
IMONTH                  0
IDAY                    0
IYEAR                   0
DISPCODE                0
SEQNO                   0
_PSU                    0
CTELENM1           349081
PVTRESD1           349082
COLGHOUS           445113
STATERE1           349082
CELPHON1           349080
LADULT1            349082
COLGSEX1           445111
NUMADULT           349100
LANDSEX1           401436
NUMMEN             394702
NUMWOMEN           394699
RESPSLCT           392879
SAFETIME            96051
CTELNUM1            96051
CELLFON5            96052
CADULT1             96052
CELLSEX1            96053
PVTRESD3            96058
CCLGHOUS           443807
CSTATE1             96060
LANDLINE            96061
HHADULT             96061
SEXVAR                  0
GENHLTH                 3
PHYSHLTH                5
MENTHLTH                3
POORHLTH           189386
PRIMINSR                4
PERSDOC3                2
MEDCOST1    

In [9]:
blank_perc = blank_counts / record_sum * 100
with pd.option_context('display.max_rows', None):
    print(blank_perc)

_STATE               0.000000
FMONTH               0.000000
IDATE                0.000000
IMONTH               0.000000
IDAY                 0.000000
IYEAR                0.000000
DISPCODE             0.000000
SEQNO                0.000000
_PSU                 0.000000
CTELENM1            78.421906
PVTRESD1            78.422131
COLGHOUS            99.995732
STATERE1            78.422131
CELPHON1            78.421682
LADULT1             78.422131
COLGSEX1            99.995282
NUMADULT            78.426175
LANDSEX1            90.183586
NUMMEN              88.670776
NUMWOMEN            88.670102
RESPSLCT            88.261235
SAFETIME            21.578094
CTELNUM1            21.578094
CELLFON5            21.578318
CADULT1             21.578318
CELLSEX1            21.578543
PVTRESD3            21.579666
CCLGHOUS            99.702335
CSTATE1             21.580116
LANDLINE            21.580340
HHADULT             21.580340
SEXVAR               0.000000
GENHLTH              0.000674
PHYSHLTH  

In [10]:
df

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4,non_blank_count,non_blank_perc
0,1,1,2032022,2,3,2022,1100,2022000001,2022000001,1.0,...,2,0.0,1,0,1,1.0,2.0,2.0,177,0.558360
1,1,1,2042022,2,4,2022,1100,2022000002,2022000002,1.0,...,2,0.0,1,0,1,2.0,2.0,2.0,175,0.552050
2,1,1,2022022,2,2,2022,1100,2022000003,2022000003,1.0,...,2,0.0,1,0,1,BLANK,BLANK,2.0,171,0.539432
3,1,1,2032022,2,3,2022,1100,2022000004,2022000004,1.0,...,2,0.0,1,0,1,9.0,9.0,2.0,162,0.511041
4,1,1,2022022,2,2,2022,1100,2022000005,2022000005,1.0,...,1,10.0,1,140,1,BLANK,BLANK,2.0,176,0.555205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,78,11,12192022,12,19,2022,1100,2022001527,2022001527,BLANK,...,7,900.0,9,99900,9,BLANK,BLANK,1.0,177,0.558360
445128,78,11,12212022,12,21,2022,1100,2022001528,2022001528,BLANK,...,2,0.0,1,0,1,BLANK,BLANK,1.0,160,0.504732
445129,78,11,11292022,11,29,2022,1100,2022001529,2022001529,BLANK,...,7,900.0,9,99900,9,2.0,2.0,2.0,169,0.533123
445130,78,11,12082022,12,8,2022,1100,2022001530,2022001530,BLANK,...,2,0.0,1,0,1,1.0,1.0,1.0,165,0.520505


In [11]:
def process_variable(var):
    if isinstance(var, int):
        result = str(var)
    elif isinstance(var, float):
        result = str(int(var))
    else:
        result = var
    
    return result

In [12]:
new_column_names = []
for column_name in df.columns:
    if column_name in data_element_details and 'Label' in data_element_details[column_name]:
        new_column_names.append(data_element_details[column_name]['Label'])
        values_map = {key: value['Value Label'] for key, value in data_element_details[column_name]['Valid Values'].items()}
        df[column_name] = df[column_name].map(lambda x: values_map.get(process_variable(x), x))
    else:
        new_column_names.append(column_name)
    
df.columns = new_column_names

In [13]:
value_counts = df['Final Disposition'].value_counts()

fig = go.Figure(data=[go.Bar(
    x=value_counts.index,  # Unique values in the Series
    y=value_counts.values  # Frequency counts
)])

fig.update_layout(
    title='Histogram of Final Disposition',
    xaxis_title='Final Disposition',
    yaxis_title='Count'
)

fig.show()

In [14]:
df

,State FIPS Code,File Month,Interview Date,Interview Month,Interview Day,Interview Year,Final Disposition,Annual Sequence Number,Primary Sampling Unit,Correct telephone number?,...,Drink any alcoholic beverages in past 30 days,Computed drink-occasions-per-day,Binge Drinking Calculated Variable,Computed number of drinks of alcohol beverages per week,Heavy Alcohol Consumption Calculated Variable,Flu Shot Calculated Variable,Pneumonia Vaccination Calculated Variable,Ever been tested for HIV calculated variable,non_blank_count,non_blank_perc
0,Alabama,January,2032022,February,,,Completed Interview,2022000001,2022000001,"YesGo to LL.02, PVTRESD1",...,No\nNotes: ALCDAY4=888,No Drink-Occasions per day\nNotes: ALCDAY4 = 888,No\nNotes: ALCDAY4<231 and DRNK3GE5=88; or ALC...,Did not drink\nNotes: DROCDY4_=0 or AVEDRNK3=88,No\nNotes: (SEXVAR=1 or BIRTHSEX=1) and _DRNKW...,Yes\nNotes: AGE >=65 and FLUSHOT7 = 1,No\nNotes: AGE >= 65 and PNEUVAC4 = 2,No\nNotes: HIVTST7 = 2,177,0.558360
1,Alabama,January,2042022,February,,,Completed Interview,2022000002,2022000002,"YesGo to LL.02, PVTRESD1",...,No\nNotes: ALCDAY4=888,No Drink-Occasions per day\nNotes: ALCDAY4 = 888,No\nNotes: ALCDAY4<231 and DRNK3GE5=88; or ALC...,Did not drink\nNotes: DROCDY4_=0 or AVEDRNK3=88,No\nNotes: (SEXVAR=1 or BIRTHSEX=1) and _DRNKW...,No\nNotes: AGE >=65 and FLUSHOT7 = 2,No\nNotes: AGE >= 65 and PNEUVAC4 = 2,No\nNotes: HIVTST7 = 2,175,0.552050
2,Alabama,January,2022022,February,,,Completed Interview,2022000003,2022000003,"YesGo to LL.02, PVTRESD1",...,No\nNotes: ALCDAY4=888,No Drink-Occasions per day\nNotes: ALCDAY4 = 888,No\nNotes: ALCDAY4<231 and DRNK3GE5=88; or ALC...,Did not drink\nNotes: DROCDY4_=0 or AVEDRNK3=88,No\nNotes: (SEXVAR=1 or BIRTHSEX=1) and _DRNKW...,Age < 65,Age < 65\nNotes: AGE < 65,No\nNotes: HIVTST7 = 2,171,0.539432
3,Alabama,January,2032022,February,,,Completed Interview,2022000004,2022000004,"YesGo to LL.02, PVTRESD1",...,No\nNotes: ALCDAY4=888,No Drink-Occasions per day\nNotes: ALCDAY4 = 888,No\nNotes: ALCDAY4<231 and DRNK3GE5=88; or ALC...,Did not drink\nNotes: DROCDY4_=0 or AVEDRNK3=88,No\nNotes: (SEXVAR=1 or BIRTHSEX=1) and _DRNKW...,Dont know/Not Sure Or Refused/Missing,Dont know/Not Sure Or Refused/Missing\nNotes:...,No\nNotes: HIVTST7 = 2,162,0.511041
4,Alabama,January,2022022,February,,,Completed Interview,2022000005,2022000005,"YesGo to LL.02, PVTRESD1",...,Yes\nNotes: 1 <= ALCDAY4 <= 231,10.0,No\nNotes: ALCDAY4<231 and DRNK3GE5=88; or ALC...,140,No\nNotes: (SEXVAR=1 or BIRTHSEX=1) and _DRNKW...,Age < 65,Age < 65\nNotes: AGE < 65,No\nNotes: HIVTST7 = 2,176,0.555205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,Virgin Islands,November,12192022,December,,,Completed Interview,2022001527,2022001527,Missing\nNotes: QSTVER > = 20,...,Dont know/Not Sure\nNotes: ALCDAY4=777,Dont know/Not Sure Or Refused/Missing\nNotes:...,Dont know/Refused/Missing\nNotes: DRNK3GE5=77...,Dont know/Not sure/Refused/Missing\nNotes: AV...,Dont know/Refused/Missing\nNotes: ALCDAY5=777...,Age < 65,Age < 65\nNotes: AGE < 65,Yes\nNotes: HIVTST7 = 1,177,0.558360
445128,Virgin Islands,November,12212022,December,,,Completed Interview,2022001528,2022001528,Missing\nNotes: QSTVER > = 20,...,No\nNotes: ALCDAY4=888,No Drink-Occasions per day\nNotes: ALCDAY4 = 888,No\nNotes: ALCDAY4<231 and DRNK3GE5=88; or ALC...,Did not drink\nNotes: DROCDY4_=0 or AVEDRNK3=88,No\nNotes: (SEXVAR=1 or BIRTHSEX=1) and _DRNKW...,Age < 65,Age < 65\nNotes: AGE < 65,Yes\nNotes: HIVTST7 = 1,160,0.504732
445129,Virgin Islands,November,11292022,November,,,Completed Interview,2022001529,2022001529,Missing\nNotes: QSTVER > = 20,...,Dont know/Not Sure\nNotes: ALCDAY4=777,Dont know/Not Sure Or Refused/Missing\nNotes:...,Dont know/Refused/Missing\nNotes: DRNK3GE5=77...,Dont know/Not sure/Refused/Missing\nNotes: AV...,Dont know/Refused/Missing\nNotes: ALCDAY5=777...,No\nNotes: AGE >=65 and FLUSHOT7 = 2,No\nNotes: AGE >= 65 and PNEUVAC4 = 2,No